In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = 'org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0'

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [3]:
from confluent_kafka import Consumer, Producer,admin,TopicPartition,KafkaError
import confluent_kafka

In [4]:
##Create streaming context
sc = SparkContext(appName="user-event-aggregator")
spark = SparkSession(sc)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,5)

##KafkaUtil config for DirectStream
topic = "user-events-1"
zk = "10.0.75.1:2181"
brokers = "10.0.75.1:9092"
kafkaParams = {"metadata.broker.list": brokers,"auto.offset.reset": "smallest","group.Id":"user-event-consumer"}

##Producer Config
print('Initiating producer-----')
#brokers = "10.0.75.1:9092"
produce_topic = "user-summary-aggregated"
producer = Producer({"bootstrap.servers": brokers})

Initiating producer-----


In [5]:
###Producer delivery report
def delivery_report(err, msg) -> None:
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        #outMsg = msg.value().decode("utf-8")
        print(f"Delivered to ---",produce_topic)
    return None

In [6]:
# schema = StructType().add('userid', StringType(), False)\
# 					.add('type', StringType(), False)\
# 					.add('metadata', StructType().add("messageid", StringType())\
# 													.add("sent_at", LongType())\
# 													.add("timestamp", LongType())\
# 													.add("received_at", LongType())\
# 													.add("apikey", StringType())\
# 													.add("spaceid", StringType())\
# 													.add("version", StringType())\
# 					, False)\
# 					.add('event', StringType(), False)\
# 					.add('event_data', StructType().add("movieid", StringType())\
# 					, False)

In [7]:
schema = StructType(\
          [\
                StructField("userid", StringType()),\
                 StructField("type", StringType()),\
                StructField("metadata", StructType([\
                 		StructField("messageid", StringType()),\
 						StructField("sent_at", LongType()),\
 						StructField("timestamp", LongType()),\
 						StructField("received_at", LongType()),\
 						StructField("apikey", StringType()),\
 						StructField("spaceid", StringType()),\
 						StructField("version", StringType())\
                            ])),\
 				StructField("event", StringType()),\
 				StructField("event_data", StructType([\
 						StructField("movieid", StringType())\
 				]))\

         ] )\

In [8]:
def aggregateStream(rdd):
  if not rdd.isEmpty():
    df = spark.read.json(rdd,schema=schema)
    print('Aggragting the stream data')
    
    #df = df.select('userid').where(col("timestamp").isNotNull())
    df_agg = df.select('userid','metadata.sent_at','metadata.received_at')
    df_agg.show()
    df_agg.registerTempTable("events")
    ##First seen and last seen
    result=spark.sql("SELECT userid, MIN(sent_at),MAX(received_at) from events GROUP BY userid")
    result.show()
    json_result = result.toJSON().collect()
    print('---------------')
    print(json_result)
    for data in json_result:
            producer.produce(produce_topic, data, callback=delivery_report,)
            producer.poll(0)
            producer.flush()


In [9]:
##kafkaStream = KafkaUtils.createStream(ssc, zk, 'consumerjo', {'user-events-1':1})
directKafkaStream = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams)
directKafkaStream.pprint()
json_payload = directKafkaStream.map(lambda v:v[1])
json_payload.pprint()
json_payload.foreachRDD( lambda rdd: aggregateStream(rdd) )

In [10]:
ssc.start()
ssc.awaitTermination(timeout=180)
# sleep(5)
# ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2020-07-02 23:12:20
-------------------------------------------
(None, '{"userid": "user12", "type": "event", "metadata": {"messageid": "e3972cb1-afca-42cc-a390-826d4987cb73", "sent_at": 1587161075, "timestamp": 1587161075, "received_at": 1587161175, "apikey": "apikey4", "spaceid": "spaceid4", "version": "version4"}, "event": "played movie", "event_data": {"movieid": "MIMWulp"}}')
(None, '{"userid": "user66", "type": "event", "metadata": {"messageid": "ebb98859-f7ed-407f-ae9b-f4874ccb9128", "sent_at": 1578996089, "timestamp": 1578996089, "received_at": 1578996189, "apikey": "apikey3", "spaceid": "spaceid3", "version": "version3"}, "event": "played movie", "event_data": {"movieid": "MIMvcOQ"}}')
(None, '{"userid": "user96", "type": "event", "metadata": {"messageid": "06039760-8e76-4ad3-80ad-6e591f4ead67", "sent_at": 1553948893, "timestamp": 1553948893, "received_at": 1553948993, "apikey": "apikey10", "spaceid": "spaceid10", "version": "v